In [1]:
from __future__ import print_function, division
import matplotlib.pyplot as plt
import pandas as pd

from neuralnilm.data.loadactivations import load_nilmtk_activations
from neuralnilm.data.syntheticaggregatesource import SyntheticAggregateSource
from neuralnilm.data.datapipeline import DataPipeline
from neuralnilm.data.processing import DivideBy, IndependentlyCenter
from neuralnilm.data.datathread import DataThread

In [2]:
NILMTK_FILENAME = '/data/mine/vadeec/merged/ukdale.h5'
TARGET_APPLIANCE = 'kettle'
SEQ_LENGTH = 256
SAMPLE_PERIOD = 6
STRIDE = SEQ_LENGTH
WINDOWS = {
    'train': {
        1: ("2014-01-01", "2014-02-01")
    },
    'unseen_activations_of_seen_appliances': {
        1: ("2014-02-02", "2014-02-08")                
    },
    'unseen_appliances': {
        2: ("2013-06-01", "2013-06-07")
    }
}

LOADER_CONFIG = {
    'nilmtk_activations': dict(
        appliances=['kettle', 'microwave', 'washing machine'],
        filename=NILMTK_FILENAME,
        sample_period=SAMPLE_PERIOD,
        windows=WINDOWS
    )
}



In [3]:
from neuralnilm.data.stridesource import StrideSource

In [4]:
stride_source = StrideSource(
    target_appliance=TARGET_APPLIANCE,
    seq_length=SEQ_LENGTH,
    filename=NILMTK_FILENAME,
    windows=WINDOWS,
    sample_period=SAMPLE_PERIOD,
    stride=STRIDE
)

INFO:neuralnilm.data.stridesource:Loading NILMTK data...
INFO:neuralnilm.data.stridesource:Loading data for UK-DALE_building_1...
INFO:neuralnilm.data.stridesource:Loaded data from building UK-DALE_building_1 for fold train from 2014-01-01 00:00:06+00:00 to 2014-01-31 23:59:54+00:00.
INFO:neuralnilm.data.stridesource:Loading data for UK-DALE_building_2...
INFO:neuralnilm.data.stridesource:Loaded data from building UK-DALE_building_2 for fold unseen_appliances from 2013-06-01 00:00:00+01:00 to 2013-06-06 23:59:54+01:00.
INFO:neuralnilm.data.stridesource:Loading data for UK-DALE_building_1...
INFO:neuralnilm.data.stridesource:Loaded data from building UK-DALE_building_1 for fold unseen_activations_of_seen_appliances from 2014-02-02 00:00:00+00:00 to 2014-02-07 23:59:54+00:00.
INFO:neuralnilm.data.stridesource:Done loading NILMTK mains data.


In [5]:
nilmtk_activations = load_nilmtk_activations(**LOADER_CONFIG['nilmtk_activations'])

INFO:neuralnilm.data.loadactivations:Loading NILMTK activations...
INFO:neuralnilm.data.loadactivations:Loading kettle for UK-DALE_building_1...
INFO:neuralnilm.data.loadactivations:Loaded 111 kettle activations from UK-DALE_building_1.
INFO:neuralnilm.data.loadactivations:Loading microwave for UK-DALE_building_1...
INFO:neuralnilm.data.loadactivations:Loaded 114 microwave activations from UK-DALE_building_1.
INFO:neuralnilm.data.loadactivations:Loading washing machine for UK-DALE_building_1...
INFO:neuralnilm.data.loadactivations:Loaded 23 washing machine activations from UK-DALE_building_1.
INFO:neuralnilm.data.loadactivations:Loading kettle for UK-DALE_building_2...
INFO:neuralnilm.data.loadactivations:Loaded 31 kettle activations from UK-DALE_building_2.
INFO:neuralnilm.data.loadactivations:Loading microwave for UK-DALE_building_2...
INFO:neuralnilm.data.loadactivations:Loaded 31 microwave activations from UK-DALE_building_2.
INFO:neuralnilm.data.loadactivations:Loading washing mac

In [6]:
nilmtk_activations.keys()

['train', 'unseen_activations_of_seen_appliances', 'unseen_appliances']

In [7]:
from neuralnilm.data.realaggregatesource import RealAggregateSource

In [8]:
ras = RealAggregateSource(
    activations=nilmtk_activations,
    target_appliance=TARGET_APPLIANCE,
    seq_length=SEQ_LENGTH,
    filename=NILMTK_FILENAME,
    windows=WINDOWS,
    sample_period=SAMPLE_PERIOD
)

INFO:neuralnilm.data.realaggregatesource:Loading NILMTK mains...
INFO:neuralnilm.data.realaggregatesource:Loading mains for UK-DALE_building_1...
INFO:neuralnilm.data.realaggregatesource:Loaded mains data from building UK-DALE_building_1 for fold train from 2014-01-01 00:00:00+00:00 to 2014-01-31 23:59:54+00:00.
INFO:neuralnilm.data.realaggregatesource:Loading mains for UK-DALE_building_2...
INFO:neuralnilm.data.realaggregatesource:Loaded mains data from building UK-DALE_building_2 for fold unseen_appliances from 2013-06-01 00:00:00+01:00 to 2013-06-06 23:59:54+01:00.
INFO:neuralnilm.data.realaggregatesource:Loading mains for UK-DALE_building_1...
INFO:neuralnilm.data.realaggregatesource:Loaded mains data from building UK-DALE_building_1 for fold unseen_activations_of_seen_appliances from 2014-02-02 00:00:00+00:00 to 2014-02-07 23:59:54+00:00.
INFO:neuralnilm.data.realaggregatesource:Done loading NILMTK mains data.
INFO:neuralnilm.data.realaggregatesource:Found 89 sections without targ

In [9]:
ras.target_inclusion_prob = 0.5
for i in range(50):
    seq = ras.get_sequence()

In [10]:
#fig, axes = plt.subplots(2)
#axes[0].plot(seq.input)
#axes[1].plot(seq.target)
#plt.show()

In [11]:
nilmtk_activations['train']['kettle']['UK-DALE_building_1'][0]._metadata

['name', 'building', 'appliance', 'fold']

In [12]:
source = SyntheticAggregateSource(
    activations=nilmtk_activations,
    target_appliance=TARGET_APPLIANCE,
    seq_length=SEQ_LENGTH,
    allow_incomplete_target=False,
    sample_period=SAMPLE_PERIOD
)

In [13]:
FOLD = 'train'
#FOLD = 'unseen_activations_of_seen_appliances'
#FOLD = 'unseen_appliances'
seq = source.get_sequence(enable_all_appliances=True)

In [14]:
#all(seq.all_appliances.sum(axis=1) == seq.input[:, 0])

In [15]:
#fig, axes = plt.subplots(2, sharex=True)
#seq.all_appliances.plot(ax=axes[0])
#axes[1].plot(seq.input)
#fig.suptitle(FOLD)
#plt.show()

In [16]:
sample = source.get_batch(num_seq_per_batch=1024).next()

In [17]:
pipeline = DataPipeline(
    sources=[source, ras, stride_source],
    num_seq_per_batch=64,
    input_processing=[DivideBy(sample.before_processing.input.flatten().std()), IndependentlyCenter()],
    target_processing=[DivideBy(sample.before_processing.target.flatten().std())]
)

# Disaggregation

nilmtk_disag_source = NILMTKDisagSource(
    filename=NILMTK_FILENAME,
    target_appliance=TARGET_APPLIANCE,
    seq_length=SEQ_LENGTH,
    buildings=[5],
    window_per_building={},
    stride=STRIDE,
    sample_period=SAMPLE_PERIOD
)

disag_pipeline = deepcopy(pipeline)
disag_pipeline.source = nilmtk_disag_source

disaggregator = Disaggregator(
    pipeline=disag_pipeline,
    output_path=PATH  # 
)

Disagregator ideas:

* make a copy of pipeline but swap source for a NILMTKDisagSource
* NILMTKDisagSource loads all data into memory (?) and iterates over chunks of it (get seq_length from pipeline.source.seq_length)
* 

In [18]:
report = pipeline.report()
report

{'pipeline': {'input_processing': [{'divisor': 593.3847, 'name': 'DivideBy'},
   {'name': 'IndependentlyCenter'}],
  'num_seq_per_batch': 64,
  'rng_seed': None,
  'source_probabilities': [0.3333333333333333,
   0.3333333333333333,
   0.3333333333333333],
  'sources': {0: {'allow_incomplete_distractors': True,
    'allow_incomplete_target': False,
    'distractor_inclusion_prob': 0.25,
    'include_incomplete_target_in_output': True,
    'name': 'SyntheticAggregateSource',
    'num_batches_for_validation': 16,
    'rng_seed': None,
    'sample_period': 6,
    'seq_length': 256,
    'target_appliance': 'kettle',
    'target_inclusion_prob': 0.5,
    'uniform_prob_of_selecting_each_building': True},
   1: {'allow_incomplete_target': True,
    'allow_multiple_target_activations_in_aggregate': False,
    'filename': '/data/mine/vadeec/merged/ukdale.h5',
    'include_incomplete_target_in_output': True,
    'include_multiple_targets_in_output': False,
    'name': 'RealAggregateSource',
    '

In [19]:
from lasagne.layers import InputLayer, RecurrentLayer, DenseLayer, ReshapeLayer

def get_net_0(input_shape, target_shape=None):
    NUM_UNITS = {
        'dense_layer_0': 100,
        'dense_layer_1':  50,
        'dense_layer_2': 100
    }

    if target_shape is None:
        target_shape = input_shape
    
    # Define layers
    input_layer = InputLayer(
        shape=input_shape
    )
    
    # Dense layers
    dense_layer_0 = DenseLayer(
        input_layer, 
        num_units=NUM_UNITS['dense_layer_0']
    )
    dense_layer_1 = DenseLayer(
        dense_layer_0, 
        num_units=NUM_UNITS['dense_layer_1']
    )
    dense_layer_2 = DenseLayer(
        dense_layer_1, 
        num_units=NUM_UNITS['dense_layer_2']
    )
    
    # Output
    final_dense_layer = DenseLayer(
        dense_layer_2,
        num_units=target_shape[1] * target_shape[2],
        nonlinearity=None
    )
    output_layer = ReshapeLayer(
        final_dense_layer,
        shape=target_shape
    )
    return output_layer

In [20]:
from neuralnilm.net import Net

batch = pipeline.get_batch()
output_layer = get_net_0(
    batch.after_processing.input.shape, 
    batch.after_processing.target.shape
)
net = Net(output_layer)

In [21]:
from neuralnilm.trainer import Trainer
from neuralnilm.metrics import Metrics

trainer = Trainer(
    net=net,
    data_pipeline=pipeline,
    experiment_id=["1"],
    metrics=Metrics(state_boundaries=[4]),
    learning_rates={0: 1E-2},
    repeat_callbacks=[
        (1000, Trainer.validate)
    ]
)

Database already has an experiment with _id == 1. Should the old experiment be deleted (both from the database and from disk)? Or quit? [Q/d] d


INFO:neuralnilm.trainer:Deleting documents for old experiment.
INFO:neuralnilm.trainer:Directory exists = '~/temp/neural_nilm/output/1'
INFO:neuralnilm.trainer:  Deleting directory.


In [ ]:
report = trainer.report()
report['data']['activations'] = LOADER_CONFIG
from neuralnilm.utils import sanitise_dict_for_mongo
sanitise_dict_for_mongo(report)

{'_id': '1',
 'data': {'activations': {'nilmtk_activations': {'appliances': ['kettle',
     'microwave',
     'washing machine'],
    'filename': '/data/mine/vadeec/merged/ukdale.h5',
    'sample_period': 6,
    'windows': {'train': {'1': ('2014-01-01', '2014-02-01')},
     'unseen_activations_of_seen_appliances': {'1': ('2014-02-02',
       '2014-02-08')},
     'unseen_appliances': {'2': ('2013-06-01', '2013-06-07')}}}},
  'pipeline': {'input_processing': [{'divisor': 593.3847045898438,
     'name': 'DivideBy'},
    {'name': 'IndependentlyCenter'}],
   'num_seq_per_batch': 64,
   'rng_seed': None,
   'source_probabilities': [0.3333333333333333,
    0.3333333333333333,
    0.3333333333333333],
   'sources': {'0': {'allow_incomplete_distractors': True,
     'allow_incomplete_target': False,
     'distractor_inclusion_prob': 0.25,
     'include_incomplete_target_in_output': True,
     'name': 'SyntheticAggregateSource',
     'num_batches_for_validation': 16,
     'rng_seed': None,
     '

In [ ]:
trainer.fit(50000)

INFO:neuralnilm.trainer:Starting training for 50000 iterations.
INFO:neuralnilm.trainer:Iteration 0: Change learning rate to 1.0E-02
INFO:neuralnilm.trainer:Compiling train cost function...
INFO:neuralnilm.trainer:Done compiling cost function.


   Update # |  Train cost  | Secs per update | Source ID
------------|--------------|-----------------|-----------
          0 |   1.071809  |    1.430238     |   1

INFO:neuralnilm.net:Compiling deterministic output function...
INFO:neuralnilm.net:Done compiling deterministic output function.



          1 |   1.055399  |    0.060915     |   1
          2 |   0.259938  |    0.050006     |   2
          3 |   0.306064  |    0.074706     |   2
          4 |   1.230501  |    0.066232     |   0
          5 |   1.187141  |    0.191933     |   1
          6 |   1.013916  |    0.123669     |   1
          7 |   0.191955  |    0.074716     |   2
          8 |   1.337786  |    0.129204     |   0
          9 |   0.192465  |    0.057607     |   2
         10 |   1.123701  |    0.202472     |   1
         11 |   1.279556  |    0.094854     |   1
         12 |   0.765603  |    0.195416     |   1
         13 |   1.178034  |    0.050371     |   0
         14 |   0.256388  |    0.069898     |   2
         15 |   0.765505  |    0.098000     |   0
         16 |   1.104850  |    0.148255     |   1
         17 |   0.239071  |    0.055918     |   2
         18 |   0.805824  |    0.184585     |   1
         19 |   1.000085  |    0.132294     |   1
         20 |   1.017266  |    0.081101     |   0

/usr/local/lib/python2.7/dist-packages/sklearn/metrics/classification.py:958: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python2.7/dist-packages/sklearn/metrics/classification.py:958: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
INFO:neuralnilm.trainer:Iteration 89: Finished training epoch


...


        347 |   0.149021  |    0.048339     |   2
        348 |   0.130940  |    0.056453     |   2
        349 |   0.701593  |    0.181864     |   1
        350 |   0.172677  |    0.051092     |   2
        351 |   0.649998  |    0.071676     |   0
        352 |   0.623449  |    0.151178     |   1
        353 |   0.611942  |    0.127778     |   1
        354 |   0.523851  |    0.201190     |   1
        355 |   0.539638  |    0.051355     |   0
        356 |   0.389245  |    0.061869     |   0
        357 |   0.572179  |    0.181097     |   1
        358 |   0.676020  |    0.139908     |   1
        359 |   0.892908  |    0.133912     |   1
        360 |   0.658616  |    0.191384     |   1
        361 |   0.711302  |    0.052330     |   0
        362 |   0.195122  |    0.058448     |   2
        363 |   0.586846  |    0.085991     |   0
        364 |   0.078420  |    0.060385     |   2
        365 |   0.591659  |    0.078015     |   0
        366 |   0.837298  |    0.172617     |   1

INFO:neuralnilm.trainer:Iteration 427: Finished training epoch



        427 |   0.141281  |    0.070929     |   2
        428 |   0.114956  |    0.066457     |   2
        429 |   0.567481  |    0.159541     |   1
        430 |   0.630031  |    0.135378     |   1
        431 |   0.601650  |    0.059857     |   0
        432 |   0.543429  |    0.211668     |   1
        433 |   0.157098  |    0.056433     |   2
        434 |   0.181390  |    0.068804     |   2
        435 |   0.535192  |    0.076267     |   0
        436 |   0.065674  |    0.073929     |   2
        437 |   0.545960  |    0.161553     |   1
        438 |   0.148415  |    0.071563     |   2
        439 |   0.499666  |    0.187392     |   1
        440 |   0.609575  |    0.131169     |   1
        441 |   0.720173  |    0.134705     |   1
        442 |   0.442578  |    0.178092     |   1
        443 |   0.136002  |    0.061887     |   2
        444 |   0.530629  |    0.185138     |   1
        445 |   0.569157  |    0.150516     |   1
        446 |   0.449907  |    0.091631     |   0

INFO:neuralnilm.trainer:Iteration 532: Finished training epoch



        532 |   0.461646  |    0.052963     |   0
        533 |   0.555840  |    0.135019     |   1
        534 |   0.132102  |    0.064347     |   2
        535 |   0.108283  |    0.075049     |   2
        536 |   0.520334  |    0.067812     |   0
        537 |   0.548537  |    0.183696     |   1
        538 |   0.155868  |    0.056764     |   2
        539 |   0.367188  |    0.082414     |   0
        540 |   0.541469  |    0.173359     |   1
        541 |   0.171591  |    0.055044     |   2
        542 |   0.055466  |    0.060064     |   2
        543 |   0.482297  |    0.106326     |   0
        544 |   0.329700  |    0.201354     |   1
        545 |   0.613430  |    0.099618     |   1
        546 |   0.489423  |    0.088802     |   0
        547 |   0.480580  |    0.198976     |   1
        548 |   0.508262  |    0.129591     |   1
        549 |   0.131959  |    0.054384     |   2
        550 |   0.339365  |    0.077925     |   0
        551 |   0.548170  |    0.090518     |   0

In [ ]:
net.train_iterations